In [1]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

station = 'RRX'
year = '2022'
channels = ['BHN','BHE','BHZ']

for channel in channels:
    file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-01-01.mseed'
    stream = read(file_path)
    print(stream)

1 Trace(s) in Stream:
CI.RRX..BHN | 2022-01-01T00:00:00.019538Z - 2022-01-01T23:59:59.994538Z | 40.0 Hz, 3456000 samples
1 Trace(s) in Stream:
CI.RRX..BHE | 2022-01-01T00:00:00.019538Z - 2022-01-01T23:59:59.994538Z | 40.0 Hz, 3456000 samples
1 Trace(s) in Stream:
CI.RRX..BHZ | 2022-01-01T00:00:00.019538Z - 2022-01-01T23:59:59.994538Z | 40.0 Hz, 3456000 samples


In [2]:
stream[0].stats

         network: CI
         station: RRX
        location: 
         channel: BHZ
       starttime: 2022-01-01T00:00:00.019538Z
         endtime: 2022-01-01T23:59:59.994538Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 3456000
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 9879, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 5058048})

In [3]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

def format_duration(duration):
    """Convert duration from seconds to hours:minutes format."""
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

def save_processed_trace(trace, original_path):
    """Save the processed trace to a new file with '_processed' appended to the original name."""
    directory, filename = os.path.split(original_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_processed{ext}"
    new_path = os.path.join(directory, new_filename)
    trace.write(new_path, format='MSEED')
    logging.info(f"Saved processed trace to: {new_path}")

def process_waveform(file_path):
    try:
        stream = read(file_path)
        #print(f"Number of traces: {len(stream)}")
        if len(stream) > 1:
            print(stream)
        
        
        stream.merge(method=1, fill_value=0)
        duration = stream[0].stats.endtime - stream[0].stats.starttime
        duration_str = format_duration(duration)
        logging.info(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")
        #print(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")

        # Save the processed trace
        save_processed_trace(stream[0], file_path)

    except Exception as e:
        logging.error(f"Error processing file {stream[0]}: {str(e)}")
        print(f"Error processing file {stream[0]}: {str(e)}")



base_dir = f"/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}"

logging.basicConfig(filename=f"{year}_{station}_trace_preprocessing.log", level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')
# Base directory to start searching


print(f'processing year {year}')
# Collect all mseed files recursively in a list
file_list = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".mseed"):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Sort the list of files alphabetically
file_list.sort()
print(f"In total {len(file_list)} files")

# Process each file that has not been processed yet, total 1095 in 2021
for file_path in file_list:
    # Skip already processed files
    if "_processed" in os.path.basename(file_path):
        logging.info(f"Skipping already processed file: {os.path.basename(file_path)}")
        continue
    
    #print(f"Processing file {file_path}")
    logging.info(f"Processing file {os.path.basename(file_path)}")
    process_waveform(file_path)

processing year 2022
In total 1095 files
2 Trace(s) in Stream:
CI.RRX..BHE | 2022-01-13T00:00:00.019538Z - 2022-01-13T23:33:29.269538Z | 40.0 Hz, 3392371 samples
CI.RRX..BHE | 2022-01-13T23:37:48.069538Z - 2022-01-13T23:59:59.994538Z | 40.0 Hz, 53278 samples
2 Trace(s) in Stream:
CI.RRX..BHE | 2022-01-14T00:00:00.019538Z - 2022-01-14T00:07:03.869538Z | 40.0 Hz, 16955 samples
CI.RRX..BHE | 2022-01-14T00:09:22.569538Z - 2022-01-14T23:59:59.994538Z | 40.0 Hz, 3433498 samples
2 Trace(s) in Stream:
CI.RRX..BHE | 2022-02-23T00:00:00.019538Z - 2022-02-23T17:27:43.794538Z | 40.0 Hz, 2514552 samples
CI.RRX..BHE | 2022-02-23T17:29:09.194539Z - 2022-02-23T23:59:59.994539Z | 40.0 Hz, 938033 samples
9 Trace(s) in Stream:
CI.RRX..BHE | 2022-05-25T00:00:00.019538Z - 2022-05-25T19:59:58.394538Z | 40.0 Hz, 2879936 samples
CI.RRX..BHE | 2022-05-25T20:00:07.594538Z - 2022-05-25T20:00:41.369538Z | 40.0 Hz, 1352 samples
CI.RRX..BHE | 2022-05-25T20:00:49.519538Z - 2022-05-25T20:03:12.169538Z | 40.0 Hz, 5707

In [4]:
file_path = '01_Data/01_Seismic_Wave_Data/2024/SDD/BHZ/SDD_BHZ_2024-11-21.mseed'

stream = read(file_path)
stream.merge()
stream

FileNotFoundError: [Errno 2] No such file or directory: '01_Data/01_Seismic_Wave_Data/2024/SDD/BHZ/SDD_BHZ_2024-11-21.mseed'